In [46]:
%pip install kaiwu-1.0.3-py3-none-any.whl
%pip install sklearn
%pip install pyqubo
%pip install dwave.system
%pip install scipy

Processing ./kaiwu-1.0.3-py3-none-any.whl
kaiwu is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated 

In [47]:
import kaiwu as kw
kw.license.init(user_id="72317291601100802", sdk_code="vDSsMrcS1XvoHxrKEyWGPu3y6bydtx")
from kaiwu.classical import SimulatedAnnealingOptimizer
import numpy as np
from scipy.optimize import minimize


Baseline model

In [49]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

# Demand data (monthly time series)
demand = np.array([9000, 9400, 9594, 9859, 9958, 10043, 10309, 10512, 10588])

# Prepare input (X) and output (y) for training
# X will consist of lagged values (autoregressive approach)
lags = 2  # Use the last two months to predict the next month
X = np.array([demand[i:i + lags] for i in range(len(demand) - lags)])
y = demand[lags:]  # Target variable is the next month's demand

# Train/Test split
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Polynomial regression for more flexibility
degree = 2  # Degree of the polynomial
poly = PolynomialFeatures(degree)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Train the model
model = LinearRegression()
model.fit(X_train_poly, y_train)

# Predict on test data
y_pred = model.predict(X_test_poly)

# Calculate error metrics
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Data: {mse:.2f}")

# Predict the next value (October demand)
latest_lag = demand[-lags:]  # Use the last two months as input
next_input = poly.transform([latest_lag])  # Transform using PolynomialFeatures
predicted_y = model.predict(next_input)

print(f"Predicted Demand for October: {predicted_y[0]:.2f}")


Mean Squared Error on Test Data: 5418084.88
Predicted Demand for October: 14290.38


Baseline Model

In [ ]:
# Historical demand data (January to September)
demand = np.array([9000, 9400, 9594, 9859, 9958, 10043, 10309, 10512, 10588])

# Define AR(2) model parameters (to be optimized)
def ar_model(params, demand):
    c, phi1, phi2 = params
    predicted = []
    for t in range(2, len(demand)):
        y_t = c + phi1 * demand[t - 1] + phi2 * demand[t - 2]
        predicted.append(y_t)
    return np.array(predicted)

# Objective function: minimize squared error
def objective(params, demand):
    c, phi1, phi2 = params
    predicted = ar_model(params, demand)
    observed = demand[2:]  # starting from index 2 due to AR(2) model
    error = np.sum((predicted - observed) ** 2)
    return error

# Initial guess for parameters [c, phi1, phi2]
initial_guess = [0, 0, 0]

# Solve using a classical optimizer (to later transform into QUBO logic)
result = minimize(objective, initial_guess, args=(demand,), method='Nelder-Mead')
c_opt, phi1_opt, phi2_opt = result.x

# Predict demand for October using the optimized AR(2) model
october_demand = c_opt + phi1_opt * demand[-1] + phi2_opt * demand[-2]

c_opt, phi1_opt, phi2_opt, october_demand


(-1.3954764576209415,
 1.052336371794449,
 -0.03604914801604513,
 10761.793384157338)

In [51]:
# Calculate Mean Squared Error (MSE) for the optimized AR(2) model
def calculate_mse(params, demand):
    predicted = ar_model(params, demand)
    observed = demand[2:]  # observed values starting from index 2
    mse = np.mean((predicted - observed) ** 2)
    return mse

# Compute MSE for the optimized parameters
mse = calculate_mse([c_opt, phi1_opt, phi2_opt], demand)
mse

6240.063117704558

Kaiwu Model

In [52]:
# Define the input data
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'June', 'Jul', 'Aug', 'Sept']
demands = np.array([9000, 9400, 9594, 9859, 9958, 10043, 10309, 10512, 10588])

# Calculate first differences to make the series more stationary
diff_demands = np.diff(demands)

# We'll use AR(2) model, so we need to transform this into a QUBO problem
# For each coefficient φ₁ and φ₂, we'll use 8 binary variables to represent 
# values between -1 and 1 with 2⁻⁸ precision

def binary_to_float(binary_vars, min_val=-1, max_val=1):
    """Convert binary variables to float in range [min_val, max_val]"""
    weights = 2.0 ** -np.arange(1, len(binary_vars) + 1)
    return min_val + (max_val - min_val) * np.sum(weights * binary_vars)

# Create QUBO variables for φ₁ and φ₂
n_bits = 8
phi1_vars = kw.qubo.ndarray(n_bits, 'phi1', kw.qubo.binary)
phi2_vars = kw.qubo.ndarray(n_bits, 'phi2', kw.qubo.binary)

# Construct objective function (Sum of Squared Errors)
obj = 0
for t in range(2, len(diff_demands)):
    # Predicted value using AR(2) model
    pred_t = (binary_to_float(phi1_vars) * diff_demands[t-1] + 
             binary_to_float(phi2_vars) * diff_demands[t-2])
    
    # Add squared error term
    error = pred_t - diff_demands[t]
    obj += error * error

# Parse QUBO
obj = kw.qubo.make(obj)

# Convert to QUBO matrix
qubo_matrix = kw.qubo.qubo_model_to_qubo_matrix(obj)['qubo_matrix']

# Solve using simulated annealing
worker = kw.classical.SimulatedAnnealingOptimizer(
    initial_temperature=100,
    alpha=0.99,
    cutoff_temperature=0.001,
    iterations_per_t=10,
    size_limit=100
)
output = worker.solve(qubo_matrix)

# Get optimal solution
opt = kw.sampler.optimal_sampler(qubo_matrix, output, bias=0)
best_solution = opt[0][0]

# Manually create solution dictionary
sol_dict = {}
for i in range(n_bits):
    sol_dict[f'phi1[{i}]'] = best_solution[i]
    sol_dict[f'phi2[{i}]'] = best_solution[i + n_bits]

# Extract coefficients from binary solutions
phi1_binary = np.array([sol_dict[f'phi1[{i}]'] for i in range(n_bits)])
phi2_binary = np.array([sol_dict[f'phi2[{i}]'] for i in range(n_bits)])

phi1 = binary_to_float(phi1_binary)
phi2 = binary_to_float(phi2_binary)

# Predict October demand
last_diff = demands[-1] - demands[-2]
second_last_diff = demands[-2] - demands[-3]
predicted_diff = phi1 * last_diff + phi2 * second_last_diff
october_demand = demands[-1] + predicted_diff

print(f"AR(2) Coefficients: φ₁ = {phi1:.4f}, φ₂ = {phi2:.4f}")
print(f"Predicted demand for October: {october_demand:.0f}")

/var/folders/92/zd0pd5bj15qbwvj0zp4_zt540000gn/T/ipykernel_44799/1043707881.py:19: DeprecationWarning: Call to deprecated function (or staticmethod) binary. (You can use the standard function `kw.qubo.Binary`.) -- Deprecated since version 1.0.0.
  phi1_vars = kw.qubo.ndarray(n_bits, 'phi1', kw.qubo.binary)
/var/folders/92/zd0pd5bj15qbwvj0zp4_zt540000gn/T/ipykernel_44799/1043707881.py:20: DeprecationWarning: Call to deprecated function (or staticmethod) binary. (You can use the standard function `kw.qubo.Binary`.) -- Deprecated since version 1.0.0.
  phi2_vars = kw.qubo.ndarray(n_bits, 'phi2', kw.qubo.binary)


AR(2) Coefficients: φ₁ = 0.9141, φ₂ = 0.9766
Predicted demand for October: 10856


In [54]:
# Generate predictions for the AR(2) model and calculate the MSE
squared_errors = []  # To store squared errors for MSE calculation

for t in range(2, len(diff_demands)):
    # Predicted value using the learned AR(2) coefficients
    predicted_diff = phi1 * diff_demands[t - 1] + phi2 * diff_demands[t - 2]
    
    # Actual value
    actual_diff = diff_demands[t]
    
    # Compute squared error
    squared_error = (predicted_diff - actual_diff) ** 2
    squared_errors.append(squared_error)

# Calculate the Mean Squared Error
mse = np.mean(squared_errors)
print(f"Mean Squared Error (MSE) of the AR(2) model: {mse:.4f}")

Mean Squared Error (MSE) of the AR(2) model: 72043.1868
